In [12]:
import pandas as pd
import numpy as np

datapath="../data/"

submission_data = pd.read_csv(datapath + "submission.csv")
test_data = pd.read_csv(datapath + "test.csv")
train_data = pd.read_csv(datapath + "train.csv")
countrycodes=pd.read_csv(datapath+"wikipedia-iso-country-codes.csv")
indicator_data=pd.read_excel(datapath+"Indicator_list.xlsx").fillna("")
age = pd.read_excel(datapath+"API_SP.POP.65UP.TO.ZS_DS2_en_excel_v2_887753.xls")
population = pd.read_excel(datapath+"total_population.xls")
population_dens = pd.read_excel(datapath+"population_density.xls")

indicator_data["long_form"]=indicator_data["long_form"].apply(lambda x: "_".join(x.split(" ")))
train_data[["ConfirmedCases","Fatalities"]]=train_data[["ConfirmedCases","Fatalities"]].astype(int)
submission_data[["ConfirmedCases","Fatalities"]]=submission_data[["ConfirmedCases","Fatalities"]].astype(int)

In [13]:
data_sets = [(age,"Population ages 65 and above"), (population,"Pop"), (population_dens,"Pop_dens")]
world= []
for worldbank, name in data_sets:
    worldbank.columns = worldbank.iloc[2]
    worldbank.drop(index=[0, 1,2],inplace = True)
    #worldbank["Year"] = 2018
    #worldbank = worldbank[["Indicator Code", "Country Code", "Year", 2018.0]]
    #worldbank.columns = ["Indicator Code", "Shortcut", "Year", name]
    worldbank = worldbank[["Country Code", 2018.0]]
    worldbank.columns = ["Shortcut", name]
    worldbank.head()
    world.append(worldbank)

In [14]:
import requests
from tqdm import tqdm

def import_data(indicator="",indicator_type="",name=""):
    url="https://ghoapi.azureedge.net/api/{}".format(indicator)
    data = pd.DataFrame(requests.get(url).json()["value"])
    data.drop([col for col in data.columns if data[col].isna().sum() == data.shape[0]],axis=1,inplace=True)
    data=data.drop(data[data.NumericValue.isna()].index,axis=0).reset_index(drop=True)
    data=data[data.Dim1==indicator_type] if len(indicator_type)>0 else data
    data=data[["IndicatorCode","SpatialDim","TimeDim","NumericValue"]].drop_duplicates(subset=["IndicatorCode","SpatialDim"],keep="last").reset_index(drop=True)
    data=data[["SpatialDim","NumericValue"]]
    data.columns=["Shortcut",name]
    
    return data

def import_who_indicators(indicator_data):
    data=[import_data(indicator["short_form"],indicator["addition"],indicator["long_form"]) for _,indicator in tqdm(indicator_data.iterrows())]
    return data
    
who_data_list=import_who_indicators(indicator_data)

for info in world:
    who_data_list.append(info)

11it [00:12,  1.13s/it]


In [15]:
def match_country_code(countrycodes,traindata):
    #countrycodes=countrycodes[["ISO3166-1-Alpha-3","official_name_en"]]
    #countrycodes=countrycodes.drop(countrycodes[countrycodes["ISO3166-1-Alpha-3"].isna()].index,axis=0).reset_index(drop=True)
    #countrycodes.columns = ["Shortcut", "Country_Region"]
    countrycodes=countrycodes[["Alpha-3 code", "English short name lower case"]]
    countrycodes=countrycodes.drop(countrycodes[countrycodes["English short name lower case"].isna()].index,axis=0).reset_index(drop=True)
    countrycodes.columns = ["Shortcut", "Country_Region"]
    result = traindata.merge(countrycodes,on="Country_Region",how="left").reset_index(drop=True)
    return result

new_traindata = match_country_code(countrycodes,train_data)

def match_dataframes(new_traindata,who_data):
    result = new_traindata.merge(who_data,on="Shortcut",how="left")
    return result

test_traindata = new_traindata.copy()
for who_data in tqdm(who_data_list):
    test_traindata=match_dataframes(test_traindata,who_data)

100%|██████████| 14/14 [00:00<00:00, 125.02it/s]


In [16]:
test_traindata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19110 entries, 0 to 19109
Data columns (total 21 columns):
Id                                                                                         19110 non-null int64
Province_State                                                                             8125 non-null object
Country_Region                                                                             19110 non-null object
Date                                                                                       19110 non-null object
ConfirmedCases                                                                             19110 non-null int32
Fatalities                                                                                 19110 non-null int32
Shortcut                                                                                   18980 non-null object
Population_using_at_least_basic_sanitation_services_(%)                                    18850 non-nul

In [18]:
test_traindata[new_traindata["Shortcut"].isna()]["Country_Region"].unique()

array(['Diamond Princess', 'Holy See'], dtype=object)